This notebook initializes some quibits to 0 and then measures them, as a test for measurement errors.

In [1]:
## Initial setup
import qiskit
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2 as Sampler
import os
from dotenv import load_dotenv

load_dotenv()
API_TOKEN = os.getenv("API_TOKEN")
service = QiskitRuntimeService(channel="ibm_quantum", token=API_TOKEN)
backend = service.backend(name="ibm_rensselaer")

In [2]:
# 2 qubit system
qc = qiskit.QuantumCircuit(2, 2)

# measure the qubits
qc.measure([0, 1], [0, 1])
qc = qiskit.transpile(qc, backend)

sampler = Sampler(backend)
job = sampler.run([qc], shots=10**4)
result = job.result()[0]

In [6]:
# display
counts = result.data.c.get_counts()
print("Measurement output: ", counts)


Measurement output:  {'00': 9945, '10': 34, '01': 21}


In [7]:
# Again, but this time with 20 qubits
qc_20 = qiskit.QuantumCircuit(20, 20)
qc_20.measure([i for i in range(20)], [i for i in range(20)])
qc_20 = qiskit.transpile(qc_20, backend)

sampler = Sampler(backend)
job_20 = sampler.run([qc_20], shots=10**4)
result_20 = job_20.result()[0]

In [8]:
counts_20 = result_20.data.c.get_counts()
print("Measurement output: ", counts_20)

Measurement output:  {'00000000000000000000': 9434, '00000000000000010000': 19, '00000001000000000000': 29, '00000000000001000000': 33, '00100000000000000000': 34, '00001000000000000000': 49, '00000000000000001000': 23, '00000010000000000000': 26, '00000100000000000000': 36, '00000000000100000000': 42, '00000000000000100000': 28, '00000000001000000000': 18, '01000000000000000000': 36, '00000000000000000010': 37, '11000000000000000000': 1, '00000000000000000001': 15, '00000000010000000000': 20, '00000000000000000100': 17, '00000000100000000000': 30, '00000000000100010000': 1, '00010000000000000000': 20, '01000100000000000000': 3, '00000000000010000000': 21, '10000000000000000000': 18, '00000000000000010001': 1, '10001000000000000000': 1, '00000000000100000100': 1, '00001010000000000000': 1, '10000000000000000001': 1, '00100000010000000000': 1, '00000000010001000000': 1, '00001000000000000001': 1, '00000000001000001000': 1, '00000001000000000001': 1}


In [11]:
counts.get('0'*20, 0)

0

In [12]:
num_errors = {}
for measurement, count in counts_20.items():
    errors = sum([1 for i in range(20) if measurement[i] != '0'])
    num_errors[errors] = num_errors.get(errors, 0) + count
print(num_errors)

{2: 400, 3: 40, 1: 2409, 4: 1, 0: 7150}


Now, let's see if anything is different if we initialize cubits to 1 instead of 0.

In [9]:
qc_20_1 = qiskit.QuantumCircuit(20, 20)
for i in range(20):
    qc_20_1.initialize([1, 0], i)

qc_20_1.measure([i for i in range(20)], [i for i in range(20)])
qc_20_1 = qiskit.transpile(qc_20_1, backend)

sampler = Sampler(backend)
job_20_1 = sampler.run([qc_20_1], shots=10**4)
result_20_1 = job_20_1.result()[0]

In [11]:
counts_20_1 = result_20_1.data.c.get_counts()
print("Measurement output: ", counts_20_1)

Measurement output:  {'00000000000000000000': 9714, '00000000000100000000': 33, '00001000000000000000': 25, '00000100000000000000': 5, '00000000000000100000': 15, '00010000000000000000': 15, '00000010000000000000': 16, '01000000000000000000': 29, '00100000000000000000': 11, '00000000000001000000': 22, '00000001000000000000': 13, '00000000000000000010': 17, '00000000001000000000': 13, '00000000000100000010': 1, '00001010000000000000': 1, '00000000000000001000': 8, '00000000000010000000': 8, '00000000000000000100': 15, '00000000000000010000': 9, '10000000000000000000': 20, '01100000000000000000': 1, '00000000100000000000': 4, '00000000010000000000': 2, '00000000000000000001': 2, '01000001000000000000': 1}
